# Regression

> In this notebook we will cover the preprocessing of a tabular data and will use following algorithms:

> 1. Linear Regression
> 2. Ridge Regression
> 3. LASSO Regression
> 4. Support Vector Machine
> 5. Decision Tree Regressor
> 6. Random Forest Regressor
> 7. LightGBM
> 8. CatBoost

---

# Data
https://www.kaggle.com/datasets/vedavyasv/usa-housing
> We are going to use the `USA_Housing` dataset. Since house price is a continuous variable, this is a regression problem. The data contains the following columns:

> * '`Avg. Area Income`': Avg. The income of residents of the city house is located in.
> * '`Avg. Area House Age`': Avg Age of Houses in the same city
> * '`Avg. Area Number of Rooms`': Avg Number of Rooms for Houses in the same city
> * '`Avg. Area Number of Bedrooms`': Avg Number of Bedrooms for Houses in the same city
> * '`Area Population`': The population of city house is located in
> * '`Price`': Price that the house sold at
> * '`Address`': Address for the house




#  Import Libraries

In [5]:
from IPython.display import clear_output

!pip install dask[dataframe] catboost

clear_output()


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os
import zipfile
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

%matplotlib inline

##  Read the Data

In [ ]:

# Download latest version
path = kagglehub.dataset_download("aariyan101/usa-housingcsv")

print("Path to dataset files:", path)


In [ ]:
csv_path = os.path.join(path, "USA_Housing.csv")

USAhousing = pd.read_csv(csv_path)
USAhousing.head()

In [ ]:
USAhousing.info()

In [ ]:
USAhousing.describe()

# 📊 Exploratory Data Analysis (EDA)

Let's create some simple plots to check out the data!

In [ ]:
sns.pairplot(USAhousing)

> * Notice that the relationship between "Price" and "Avg. Area Income" shows a clear positive correlation: higher income tends to be associated with higher property prices.
> * Also, the scatter plots between "Avg. Area Number of Bedrooms" and other variables show horizontal/vertical stripes, suggesting this variable is categorical or has fixed values (discrete).

In [ ]:
sns.heatmap(USAhousing.drop("Address",axis=1).corr(), annot=True)

#  Features Engineering

> We have one text column in the data (i.e. Address). Maybe we can extract the town/city name from it and use it as a feature.

In [ ]:
# Extract city/town from the "Address" column (Not optimal, we just take the first word after the \n. You can see many mistakes)
USAhousing['City/Town'] = USAhousing['Address'].str.split('\n').str[1].str.split(',').str[0]
USAhousing

> Let's add some interaction features.

In [ ]:
# Wealthier areas with more rooms might be associated with higher property prices.
USAhousing['Income_per_Room'] = USAhousing['Avg. Area Income'] / USAhousing['Avg. Area Number of Rooms']
# Combine population and number of rooms to estimate density.
USAhousing['Population_per_Room'] = USAhousing['Area Population'] / USAhousing['Avg. Area Number of Rooms']
# Room-to-Bedroom Ratio: Might helps identify homes with better layouts.
USAhousing['Room_to_Bedroom_Ratio'] = USAhousing['Avg. Area Number of Rooms'] / USAhousing['Avg. Area Number of Bedrooms']
# Age-to-Income Ratio: Indicates older homes in wealthier areas.
#############################
#TODO, add "Age_to_Income_Ratio"


#############################

> * Aggregations are one of the most powerful feature engineering techniques, as they summarize group-level patterns
> * E.g. average of income per city.

In [ ]:
# Calculate average and median for selected columns for each City/Town
city_aggregates = USAhousing.groupby('City/Town').agg({
    'Avg. Area Income': ['mean', 'median'],
    'Avg. Area House Age': ['mean', 'median'],
    'Avg. Area Number of Rooms': ['mean', 'median'],
    'Area Population': ['mean', 'median']
})
city_aggregates

In [ ]:
# Rename the columns for clarity
city_aggregates.columns = ['Income_avg_per_city', 'Income_median_per_city',
                           'HouseAge_avg_per_city', 'HouseAge_median_per_city',
                           'Rooms_avg_per_city', 'Rooms_median_per_city',
                           'Population_avg_per_city', 'Population_median_per_city']

# Merge the aggregated data back into the original dataset
USAhousing = USAhousing.merge(city_aggregates, on='City/Town', how='left')

# Display the first few rows of the updated dataset
USAhousing

#  Data Preprocessing

> Let's check the missing values.
>
> Also we don't need the Address, so will drop it.

In [17]:
USAhousing = USAhousing.drop("Address",axis=1)

In [ ]:
#############################
# TODO, report the missing values if any
#missing_values = ??
#############################
print("Missing values in each column:")
print(missing_values)

> The City/Town column is categorical. We should convert it to numerical before inputting it into the model.

> We will use LabelEncoder from sklearn, which should give each category a unique number.



In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the City/Town column
USAhousing['City/Town'] = label_encoder.fit_transform(USAhousing['City/Town'])
USAhousing

> Linear models and SVM don't work well without scaling the values.
>
> We will use MinMaxScaler from sklearn, which would convert the values to range [0,1]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# We want to scale all the columns, BUT NOT THE TARGET
cols = USAhousing.columns.drop("Price") # Ask the students why?

# Fit and transform the City/Town column
USAhousing[cols] = scaler.fit_transform(USAhousing[cols])
USAhousing

#  Training our Regression Models

> We will need to first split up our data into an X array that contains the features to train on, and a y array with the target variable, in this case, the Price column.

In [21]:
#############################
# TODO,
#X = ??
#y = ??
#############################

> We will use KFold to evaluate our models

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error


models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=10, positive=True, max_iter=10000),
    "LASSO Regression": Lasso(alpha=100, positive=True, max_iter=10000),
    "Support Vector Machine": SVR(kernel='linear'),
    "Decision Tree Regressor": DecisionTreeRegressor(max_depth=5),
    "Random Forest Regressor": RandomForestRegressor(n_estimators=100),
    "LightGBM": LGBMRegressor(verbose=-1),
    "CatBoost": CatBoostRegressor(verbose=0)
}


for model_name, model in models.items():
    scores_mae = []
    scores_rmse = []

    # 5-Fold cross-validation
    for train_index, test_index in KFold(n_splits=5).split(X, y):
        # Split data into training and testing sets
        X_Train, X_Test = X.loc[train_index,:], X.loc[test_index,:]
        y_Train, y_Test = y[train_index], y[test_index]
        # Train the model
        model.fit(X_Train, y_Train)
        # Predict on the test set
        y_pred = model.predict(X_Test)
        # Calculate metrics
        scores_mae.append(mean_absolute_error(y_Test, y_pred))
        scores_rmse.append(np.sqrt(mean_squared_error(y_Test, y_pred)))
    # Print the results
    print(f"{model_name} MAE Score: {np.mean(scores_mae)}")
    print(f"{model_name} RMSE Score: {np.mean(scores_rmse)}\n")

> * is RMSE=100k good? How to know if a score is good or not?
>
> * We need something to compare against. We need a **Baseline**.
>
> * If our models are better than the baseline score, then they are good.
>
> * The simplest baseline we can consider is the mean of the target.

In [ ]:
# Calculate the baseline predictions (mean of the target)
baseline_pred = np.full_like(y, y.mean())

# Evaluate the baseline
#############################
# TODO, find the baseline error
#baseline_mae = ??
#############################
baseline_sqrt = np.sqrt(mean_squared_error(y, baseline_pred))

print(f"Baseline MAE (using mean target): {baseline_mae:.4f}")
print(f"Baseline RMSE (using mean target): {baseline_sqrt:.4f}")

> * Yeah our models are significanlty better than the baselines (Except the SVM).
>
> * Let's Check what are the important features

In [ ]:
import matplotlib.pyplot as plt
# Retrieve Ridge coefficients and sort by absolute importance
ridge_model = models["Ridge Regression"]
ridge_importance = list(zip(X.columns, ridge_model.coef_))
sorted_ridge_importance = sorted(ridge_importance, key=lambda x: abs(x[1]), reverse=True)

# Extract sorted features and their coefficients
features, coefficients = zip(*sorted_ridge_importance)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(features, coefficients, color='darkblue')
plt.xlabel('Coefficient Value')
plt.ylabel('Features')
plt.title('Ridge Regression Feature Importance')
plt.gca().invert_yaxis()  # Invert y-axis to show the most important features at the top
plt.show()

In [ ]:
# Retrieve LASSO coefficients and sort by absolute importance
lasso_model = models["LASSO Regression"]
lasso_importance = list(zip(X.columns, lasso_model.coef_))
sorted_lasso_importance = sorted(lasso_importance, key=lambda x: abs(x[1]), reverse=True)

# Extract features and their coefficients
features, coefficients = zip(*sorted_lasso_importance)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(features, coefficients, color='darkred')
plt.xlabel('Coefficient Value')
plt.ylabel('Features')
plt.title('LASSO Regression Feature Importance')
plt.gca().invert_yaxis()  # Invert y-axis to show the most important features at the top
plt.show()

### Notice how Lasso zeroed most of the features and just focused on the most important features.
### This is what we call "sparse weights".

In [ ]:
# Retrieve CatBoost feature importances and sort them
catboost_model = models["CatBoost"]
catboost_importance = list(zip(X.columns, catboost_model.feature_importances_))
sorted_catboost_importance = sorted(catboost_importance, key=lambda x: x[1], reverse=True)

# Extract features and their importances
features, importances = zip(*sorted_catboost_importance)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(features, importances, color='orange')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('CatBoost Feature Importance')
plt.gca().invert_yaxis()  # Invert y-axis to show the most important features at the top
plt.show()


## Final Notes:
> * Gradient Boosting algorithms (e.g., LightGBM, CatBoost) often outperform other algorithms in tabular data. However, in this case, linear models outperformed others, suggesting a strong linear relationship between the features and the target.


> * Notice how LASSO eliminated many of the engineered features, indicating they might not contribute significantly to predictive performance.


> * Remember, feature engineering is an iterative process. You add new features, evaluate their impact using cross-validation (CV), and refine based on the results. What we did here is just adding features based on our intution and run the model. But it is possible that all of them are useless **(Try to rerun the notebook without the feature engineering part and check if scores improve)**.

# Created by:
[Mohamed Eltayeb](https://www.kaggle.com/mohammad2012191)
